In [1]:
import os
import requests
import json
import re
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

True

In [10]:
def obtener_api_url(clave):
    # """Obtiene la URL de la API del archivo de configuración."""
    try:
        with open("api_config.txt", "r") as f:
            for linea in f:
                if clave in linea:
                    return linea.split("=")[1].strip()
    except FileNotFoundError:
        print("Archivo de configuración no encontrado.")
        return None

def obtener_informacion_usuario(credenciales):
    # """Obtiene la información del usuario de la API de inicio de sesión."""
    url = obtener_api_url("login_api")
    if url:
        try:
            response = requests.post(url, json=credenciales)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error al obtener la información del usuario: {e}")
            return None
    else:
        return None

def obtener_categorias_faq():
    """Obtiene las categorías de preguntas frecuentes de la API."""
    url = obtener_api_url("faq_api")
    if url:
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            categorias = [item["nombre"] for item in data["values"]]
            return categorias
        except requests.exceptions.RequestException as e:
            print(f"Error al obtener las categorías de la API: {e}")
            return None
    else:
        return None
    
def obtener_documento_por_titulo(titulo):
    """Obtiene el archivo del documento por título desde la API."""
    url = obtener_api_url("document_api")
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        for documento in data["values"]:
            if documento["titulo"].lower() == titulo.lower():
                return documento["archivo"]
        return None  # Documento no encontrado
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener el documento: {e}")
        return None

def analizar_consulta_login(consulta):
    """Analiza la consulta del usuario para solicitudes de inicio de sesión y extrae las credenciales."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )

    prompt = f"""
    Determina si la siguiente consulta es una solicitud de inicio de sesión. Si es así, extrae las credenciales como un diccionario JSON.

    Consulta: "{consulta}"

    Si la consulta es una solicitud de inicio de sesión, responde con el diccionario JSON de las credenciales. De lo contrario, responde "no".
    """

    respuesta = llm.invoke(prompt).content.strip()

    # Extraer el JSON puro de la respuesta
    json_match = re.search(r'\{.*\}', respuesta, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        try:
            credenciales = json.loads(json_str)
            return credenciales
        except json.JSONDecodeError:
            print("Error al analizar el JSON extraído.")
            return None
    else:
        return None

def analizar_consulta_faq(consulta):
    """Analiza la consulta del usuario para preguntas frecuentes."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )

    prompt = f"""
    Determina si la siguiente consulta es una pregunta sobre las categorías de preguntas frecuentes:
    "{consulta}"

    Responde "sí" o "no".
    """

    respuesta = llm.invoke(prompt).content.lower()
    return "sí" in respuesta

def analizar_consulta_documento(consulta):
    """Analiza la consulta del usuario para solicitudes de documentos por título."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )

    prompt = f"""
    Determina si la siguiente consulta solicita un documento por título. Si es así, extrae el título del documento.

    Consulta: "{consulta}"

    Si la consulta solicita un documento, responde con el título del documento. De lo contrario, responde "no".
    """

    respuesta = llm.invoke(prompt).content.strip()
    return respuesta if respuesta.lower() != "no" else None

In [13]:
# query = """hola chat, deseo loguear, y mis credenciales son 
#     "client_id": "niRhPM5DtjrNc5VraICH9olcFa8ybo2LB7Yh9G3v",
#     "client_secret": "IuVL6tUcxy64hLKZRi8OmJt9E1phDXfuzfEpHxqXz11wdsoRcUOmA4V1m8gNPaMziNjl9In2jQmjr1t07ognpcBjmpNLTXwGqCPlUkpVGXc8h9ybd33D2tYjV5NlNvUM",
#     "username": "katherine.aguirreor@gmail.com",
#     "password": "PiraguaSI2025-"""

# query = """Cuales son las preguntas mas frecuentes"""

query = """muestrame el doucmento con titulo "Calidad del aire: reporte semanal del 29 abr al 05 may de 2024" """

credenciales = analizar_consulta_login(query)
titulo_documento = analizar_consulta_documento(query)


if credenciales:
    informacion_usuario = obtener_informacion_usuario(credenciales)
    if informacion_usuario:
        print(informacion_usuario)
    else:
        print("No se pudo obtener la información del usuario.")
elif titulo_documento:
    archivo_documento = obtener_documento_por_titulo(titulo_documento)
    if archivo_documento:
        print(f"El archivo del documento es: {archivo_documento}")
    else:
        print("Documento no encontrado.")
elif analizar_consulta_faq(query):
    categorias = obtener_categorias_faq()
    if categorias:
        respuesta = "Las categorías de preguntas frecuentes son: " + ", ".join(categorias)
        print(respuesta)
    else:
        print("No se pudo obtener la información de las categorías.")
else:
    print("La consulta no es una solicitud válida.")

Documento no encontrado.


In [ ]:
# query = """hola chat, deseo loguear, y mis credenciales son 
#     "client_id": "niRhPM5DtjrNc5VraICH9olcFa8ybo2LB7Yh9G3v",
#     "client_secret": "IuVL6tUcxy64hLKZRi8OmJt9E1phDXfuzfEpHxqXz11wdsoRcUOmA4V1m8gNPaMziNjl9In2jQmjr1t07ognpcBjmpNLTXwGqCPlUkpVGXc8h9ybd33D2tYjV5NlNvUM",
#     "username": "katherine.aguirreor@gmail.com",
#     "password": "PiraguaSI2025-"""

# query = """Cuales son las preguntas mas frecuentes"""

#query = """muestrame el doucmento con titulo "Calidad del aire: reporte semanal del 29 abr al 05 may de 2024" """


In [ ]:
def obtener_api_url(clave):
    """Obtiene la URL de la API del archivo de configuración."""
    try:
        with open("api_config.txt", "r") as f:
            for linea in f:
                if clave in linea:
                    return linea.split("=")[1].strip()
    except FileNotFoundError:
        print("Archivo de configuración no encontrado.")
        return None

def obtener_informacion_usuario(credenciales):
    """Obtiene la información del usuario de la API de inicio de sesión."""
    url = obtener_api_url("login_api")
    if url:
        try:
            response = requests.post(url, json=credenciales)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return f"Error al obtener la información del usuario: {e}"
    return "No se pudo obtener la información del usuario."

def obtener_categorias_faq():
    """Obtiene las categorías de preguntas frecuentes de la API."""
    url = obtener_api_url("faq_api")
    if url:
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            categorias = [item["nombre"] for item in data["values"]]
            return "Las categorías de preguntas frecuentes son: " + ", ".join(categorias) if categorias else "No se encontró información de categorías."
        except requests.exceptions.RequestException as e:
            return f"Error al obtener las categorías: {e}"
    return "No se pudo obtener la información de las categorías."

def obtener_documento_por_titulo(titulo):
    """Obtiene el archivo del documento por título desde la API."""
    url = obtener_api_url("document_api")
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        for documento in data["values"]:
            if documento["titulo"].lower() == titulo.lower():
                return f"El archivo del documento es: {documento['archivo']}"
        return "Documento no encontrado."
    except requests.exceptions.RequestException as e:
        return f"Error al obtener el documento: {e}"

def analizar_consulta_login(consulta):
    """Analiza la consulta del usuario para solicitudes de inicio de sesión y extrae las credenciales."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )

    prompt = f"""
    Determina si la siguiente consulta es una solicitud de inicio de sesión. Si es así, extrae las credenciales como un diccionario JSON.

    Consulta: "{consulta}"

    Si la consulta es una solicitud de inicio de sesión, responde con el diccionario JSON de las credenciales. De lo contrario, responde "no".
    """

    respuesta = llm.invoke(prompt).content.strip()

    # Extraer el JSON puro de la respuesta
    json_match = re.search(r'\{.*\}', respuesta, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        try:
            credenciales = json.loads(json_str)
            return credenciales
        except json.JSONDecodeError:
            print("Error al analizar el JSON extraído.")
            return None
    else:
        return None

def analizar_consulta_faq(consulta):
    """Analiza la consulta del usuario para preguntas frecuentes."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )

    prompt = f"""
    Determina si la siguiente consulta es una pregunta sobre las categorías de preguntas frecuentes:
    "{consulta}"

    Responde "sí" o "no".
    """

    respuesta = llm.invoke(prompt).content.lower()
    return "sí" in respuesta

def analizar_consulta_documento(consulta):
    # """Analiza la consulta del usuario para solicitudes de documentos por título."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )

    prompt = f"""
    Determina si la siguiente consulta solicita un documento por título. Si es así, extrae el título del documento.

    Consulta: "{consulta}"

    Si la consulta solicita un documento, responde con el título del documento. De lo contrario, responde "no".
    """

    respuesta = llm.invoke(prompt).content.strip()
    return respuesta if respuesta.lower() != "no" else None

def manejar_consulta(consulta):
    """Maneja la consulta del usuario y llama a la función apropiada."""
    if credenciales := analizar_consulta_login(consulta):
        return obtener_informacion_usuario(credenciales)
    elif titulo_documento := analizar_consulta_documento(consulta):
        return obtener_documento_por_titulo(titulo_documento)
    elif analizar_consulta_faq(consulta):
        return obtener_categorias_faq()
    else:
        return "La consulta no es una solicitud válida."

# query = """muestrame el doucmento con titulo "Calidad del aire: reporte semanal del 06 al 12 de mayo de 2024" """

query = """hola chat, deseo loguear, 
    "client_id": "niRhPM5DtjrNc5VraICH9olcFa8ybo2LB7Yh9G3v",
    "client_secret": "IuVL6tUcxy64hLKZRi8OmJt9E1phDXfuzfEpHxqXz11wdsoRcUOmA4V1m8gNPaMziNjl9In2jQmjr1t07ognpcBjmpNLTXwGqCPlUkpVGXc8h9ybd33D2tYjV5NlNvUM",
    "username": "katherine.aguirreor@gmail.com",
    "password": "PiraguaSI2025-"""


respuesta = manejar_consulta(query)
print(respuesta)

El archivo del documento es: https://djangoapi.s3.amazonaws.com/app/documentacion/F-PMA-04_Reporte_semanal_operacion_SVCA.pdf


{'oauth2': {'access_token': 'oWRdDkbERzpmdSlgkpETRG3M7XQyGg', 'expires_in': 36000, 'token_type': 'Bearer', 'scope': 'read write', 'refresh_token': 'UVqQbiqCNRHzX5TeKkzDaBik8XlHTw'}, 'id': 8019, 'user': {'id': 8875, 'username': 'katherine.aguirreor@gmail.com', 'is_active': True, 'is_staff': True, 'user_permissions': [], 'groups': [10]}, 'app_token': '', 'tipo_identificacion': 'CODIGO', 'identificacion': 'katherine.aguirreor@gmail.com', 'municipio': 1, 'territorial': 8, 'grupo_piraguero': None, 'zona': 'ND', 'nombres': '', 'apellidos': '', 'genero': 'ND', 'ocupacion': '', 'rango_edad': 'INDETERMINADO', 'fecha_nacimiento': None, 'grupo_poblacional': 'NINGUNO', 'tipo_actor': 'Sin especificar', 'nivel_academico': 'Estudiante', 'empresa': None, 'organizacion': '', 'email': 'katherine.aguirreor@gmail.com', 'telefono': '', 'foto': None, 'flags': {'id': 7028, 'lider': False, 'custodio': False, 'mide_lluvia': False, 'reporta_lluvia': False, 'gestor_limnigrafico': False, 'gestor_pluviografico': F

In [ ]:
import os
import requests
import json
import re
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

def obtener_api_url(clave):
    # Función para obtener la URL de una API desde un archivo de configuración.
    try:
        # Intenta abrir el archivo 'api_config.txt' en modo lectura.
        with open("api_config.txt", "r") as f:
            # Itera a través de cada línea del archivo.
            for linea in f:
                # Verifica si la clave proporcionada existe en la línea.
                if clave in linea:
                    # Si la clave existe, divide la línea por el signo '=' y devuelve la URL (la segunda parte de la línea).
                    return linea.split("=")[1].strip()
    except FileNotFoundError:
        # Si el archivo no se encuentra, imprime un mensaje de error y devuelve None.
        print("Archivo de configuración no encontrado.")
        return None

def obtener_informacion_usuario(credenciales):
    # Función para obtener información del usuario desde una API de inicio de sesión.
    url = obtener_api_url("login_api")
    # Obtiene la URL de la API de inicio de sesión utilizando la función 'obtener_api_url'.
    if url:
        # Verifica si se obtuvo una URL válida.
        try:
            # Intenta realizar una solicitud POST a la API con las credenciales proporcionadas.
            response = requests.post(url, json=credenciales)
            # Verifica si la respuesta de la API fue exitosa.
            response.raise_for_status()
            # Devuelve la respuesta de la API en formato JSON.
            return response.json()
        except requests.exceptions.RequestException as e:
            # Si ocurre un error al realizar la solicitud, devuelve un mensaje de error.
            return f"Error al obtener la información del usuario: {e}"
    # Si no se obtuvo una URL válida, devuelve un mensaje de error.
    return "No se pudo obtener la información del usuario."

def obtener_categorias_faq():
    # Función para obtener las categorías de preguntas frecuentes desde una API.
    url = obtener_api_url("faq_api")
    # Obtiene la URL de la API de preguntas frecuentes utilizando la función 'obtener_api_url'.
    if url:
        # Verifica si se obtuvo una URL válida.
        try:
            # Intenta realizar una solicitud GET a la API.
            response = requests.get(url)
            # Verifica si la respuesta de la API fue exitosa.
            response.raise_for_status()
            # Convierte la respuesta de la API a formato JSON.
            data = response.json()
            # Extrae los nombres de las categorías de la respuesta.
            categorias = [item["nombre"] for item in data["values"]]
            # Devuelve una cadena con las categorías o un mensaje si no se encontraron categorías.
            return "Las categorías de preguntas frecuentes son: " + ", ".join(categorias) if categorias else "No se encontró información de categorías."
        except requests.exceptions.RequestException as e:
            # Si ocurre un error al realizar la solicitud, devuelve un mensaje de error.
            return f"Error al obtener las categorías: {e}"
    # Si no se obtuvo una URL válida, devuelve un mensaje de error.
    return "No se pudo obtener la información de las categorías."

def obtener_documento_por_titulo(titulo):
    # Función para obtener el archivo de un documento por título desde una API.
    url = obtener_api_url("document_api")
    # Obtiene la URL de la API de documentos utilizando la función 'obtener_api_url'.
    try:
        # Intenta realizar una solicitud GET a la API.
        response = requests.get(url)
        # Verifica si la respuesta de la API fue exitosa.
        response.raise_for_status()
        # Convierte la respuesta de la API a formato JSON.
        data = response.json()
        # Itera a través de los documentos en la respuesta.
        for documento in data["values"]:
            # Verifica si el título del documento coincide con el título proporcionado (ignorando mayúsculas/minúsculas).
            if documento["titulo"].lower() == titulo.lower():
                # Devuelve el archivo del documento si se encuentra.
                return f"El archivo del documento es: {documento['archivo']}"
        # Si no se encuentra el documento, devuelve un mensaje.
        return "Documento no encontrado."
    except requests.exceptions.RequestException as e:
        # Si ocurre un error al realizar la solicitud, devuelve un mensaje de error.
        return f"Error al obtener el documento: {e}"

def analizar_consulta_login(consulta):
    # Función para analizar la consulta del usuario y extraer las credenciales de inicio de sesión.
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )
    # Inicializa el modelo de lenguaje de Google Generative AI.
    prompt = f"""
    Determina si la siguiente consulta es una solicitud de inicio de sesión. Si es así, extrae las credenciales como un diccionario JSON.

    Consulta: "{consulta}"

    Si la consulta es una solicitud de inicio de sesión, responde con el diccionario JSON de las credenciales. De lo contrario, responde "no".
    """
    # Define el prompt para el modelo de lenguaje.
    respuesta = llm.invoke(prompt).content.strip()
    # Envía el prompt al modelo de lenguaje y obtiene la respuesta.
    json_match = re.search(r'\{.*\}', respuesta, re.DOTALL)
    # Busca un objeto JSON en la respuesta del modelo.
    if json_match:
        # Si se encuentra un objeto JSON, intenta cargarlo como un diccionario.
        json_str = json_match.group(0)
        try:
            credenciales = json.loads(json_str)
            return credenciales
        except json.JSONDecodeError:
            # Si ocurre un error al cargar el JSON, devuelve None.
            return None
    else:
        # Si no se encuentra un objeto JSON, devuelve None.
        return None

def analizar_consulta_faq(consulta):
    # Función para analizar la consulta del usuario y determinar si es una pregunta sobre preguntas frecuentes.
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )
    # Inicializa el modelo de lenguaje de Google Generative AI.
    prompt = f"""
    Determina si la siguiente consulta es una pregunta sobre las categorías de preguntas frecuentes:
    "{consulta}"

    Responde "sí" o "no".
    """
    # Define el prompt para el modelo de lenguaje.
    respuesta = llm.invoke(prompt).content.lower()
    # Envía el prompt al modelo y verifica si la respuesta contiene "sí".
    return "sí" in respuesta

def analizar_consulta_documento(consulta):
    # Función para analizar la consulta del usuario y extraer el título del documento solicitado.
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )
    # Inicializa el modelo de lenguaje de Google Generative AI.
    prompt = f"""
    Determina si la siguiente consulta solicita un documento por título. Si es así, extrae el título del documento.

    Consulta: "{consulta}"

    Si la consulta solicita un documento, responde con el título del documento. De lo contrario, responde "no".
    """
    # Define el prompt para el modelo de lenguaje.
    respuesta = llm.invoke(prompt).content.strip()
    # Envía el prompt al modelo y devuelve el título del documento o None.
    return respuesta if respuesta.lower() != "no" else None

def manejar_consulta(consulta):
    # Función principal para manejar las consultas del usuario utilizando un diccionario de funciones.

    def manejar_login(consulta):
        # Función interna para manejar consultas de inicio de sesión.
        if credenciales := analizar_consulta_login(consulta):
            # Si se extraen las credenciales, obtiene la información del usuario.
            return obtener_informacion_usuario(credenciales)
        return None

    def manejar_documento(consulta):
        # Función interna para manejar consultas de documentos.
        if titulo_documento := analizar_consulta_documento(consulta):
            # Si se extrae el título del documento, obtiene el archivo del documento.
            return obtener_documento_por_titulo(titulo_documento)
        return None

    def manejar_faq(consulta):
        # Función interna para manejar consultas de preguntas frecuentes.
        if analizar_consulta_faq(consulta):
            # Si la consulta es sobre preguntas frecuentes, obtiene las categorías de preguntas frecuentes.
            return obtener_categorias_faq()
        return None

    funciones_consulta = {
        manejar_login: analizar_consulta_login,
        manejar_documento: analizar_consulta_documento,
        manejar_faq: analizar_consulta_faq,
    }
    # Diccionario que mapea funciones de manejo a funciones de análisis.
    for funcion, funcion_analisis in funciones_consulta.items():
        # Itera a través del diccionario de funciones.
        if funcion_analisis(consulta):
            # Si la función de análisis coincide con la consulta, llama a la función de manejo.
            resultado = funcion(consulta)
            if resultado:
                # Si la función de manejo devuelve un resultado, lo devuelve.
                return resultado
    # Si ninguna función de análisis coincide, devuelve un mensaje de error.
    return "La consulta no es una solicitud válida."

# query = """hola chat, deseo loguear, 
#     "client_id": "niRhPM5DtjrNc5VraICH9olcFa8ybo2LB7Yh9G3v",
#     "client_secret": "IuVL6tUcxy64hLKZRi8OmJt9E1phDXfuzfEpHxqXz11wdsoRcUOmA4V1m8gNPaMziNjl9In2jQmjr1t07ognpcBjmpNLTXwGqCPlUkpVGXc8h9ybd33D2tYjV5NlNvUM",
#     "username": "katherine.aguirreor@gmail.com",
#     "password": "PiraguaSI2025-"""

# query = """Cuales son todos los tipos de preguntas"""

query = """0
.......................... """

respuesta = manejar_consulta(query)
print(respuesta)

El archivo del documento es: https://djangoapi.s3.amazonaws.com/app/documentacion/Reporte_semanal_operacion_SVCA_Abril29-M.pdf
